<a href="https://colab.research.google.com/github/fpcaforio/grace/blob/master/GRACE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLS-KDD

In [1]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np
import pickle
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import time

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#CONFIG

dataset = 'UNSW-NB15'
N_CLASSES = 2
n_clusters = 1000
n_neighbors = 2

if dataset == 'NLS-KDD':
  print('Dataset: NLS-KDD')
  pathHeatmap = '/content/drive/MyDrive/Grace/NLS-KDD/heatmap/'
  pathMagneto = '/content/drive/MyDrive/Grace/NLS-KDD/magneto/'
  size = 12
elif dataset == 'UNSW-NB15':
  print('Dataset: UNSW-NB15')
  pathHeatmap = '/content/drive/MyDrive/Grace/UNSW-NB15/heatmap/'
  pathMagneto = '/content/drive/MyDrive/Grace/UNSW-NB15/magneto/'
  size = 15
elif dataset == 'KDD-CUP99':  
  print('KDD-CUP99')
  pathHeatmap = '/content/drive/MyDrive/Grace/KDD-CUP99/heatmap/'
  pathMagneto = '/content/drive/MyDrive/Grace/KDD-CUP99/magneto/'
  size = 13

Dataset: UNSW-NB15


In [5]:
f_myfile = open(pathHeatmap + 'XTrain_heatmap.pickle', 'rb')
XTrain = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open(pathMagneto + 'Ytrain.pickle', 'rb')
YTrain = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open(pathHeatmap + 'XTest_heatmap.pickle', 'rb')
XTest = pickle.load(f_myfile)
f_myfile.close()

f_myfile = open(pathMagneto + 'Ytest.pickle', 'rb')
YTest = pickle.load(f_myfile)
f_myfile.close()

XTrain = np.array(XTrain)
XTrain = np.uint8(255 * XTrain)
XTrain = XTrain / 255
XTrain = XTrain.reshape(-1, (size*size))

XTest = np.array(XTest)
XTest = np.uint8(255 * XTest)
XTest = XTest / 255
XTest = XTest.reshape(-1, (size*size))

print('Training Set: ', XTrain.shape)
print('Training Set: ', XTest.shape)

Training Set:  (82332, 225)
Training Set:  (175341, 225)


In [6]:
XTrain_normal = XTrain[np.where(YTrain[:] == 1)]
XTrain_attack = XTrain[np.where(YTrain[:] == 0)]

print('Training Set - Normal: ', XTrain_normal.shape)
print('Training Set - Attack: ', XTrain_attack.shape)

Training Set - Normal:  (37000, 225)
Training Set - Attack:  (45332, 225)


In [7]:
kmeans_normal = KMeans(n_clusters=n_clusters, random_state=0).fit(XTrain_normal)
centroids_normal = kmeans_normal.cluster_centers_

kmeans_attack = KMeans(n_clusters=n_clusters, random_state=0).fit(XTrain_attack)
centroids_attack = kmeans_attack.cluster_centers_

centroids = np.concatenate((centroids_normal, centroids_attack), axis=0)
centroidsY = np.concatenate((np.ones(n_clusters), np.zeros(n_clusters)), axis=0)

In [8]:
#KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=n_neighbors)
classifier.fit(centroids, centroidsY)
y_pred = classifier.predict(XTest)
con_mat = confusion_matrix(YTest, y_pred)

tp = con_mat[0][0]  # attacks true
fn = con_mat[0][1]  # attacs predict normal
fp = con_mat[1][0]  # normal predict attacks
tn = con_mat[1][1]  # normal as normal
attacks = tp + fn
normals = fp + tn
OA = (tp + tn) / (attacks + normals)
AA = ((tp / attacks) + (tn / normals)) / N_CLASSES
P = tp / (tp + fp)
R = tp / (tp + fn)
F1 = 2 * ((P * R) / (P + R))
FAR = fp / (fp + tn)
TPR = tp / (tp + fn)
r = (tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR)

print('tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR')
print(r)

tp, fn, fp, tn, OA, AA, P, R, F1, FAR, TPR
(116615, 2726, 9589, 46411, 0.9297654285078789, 0.9029628746167944, 0.9240198408925232, 0.9771578920907316, 0.9498462603596083, 0.17123214285714286, 0.9771578920907316)
